# GPU CONFIGURATION

In [ ]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'  # Reduce TensorFlow logging
import tensorflow as tf
from tensorflow import keras

# Initialize TensorFlow
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
    except RuntimeError as e:
        print(e)
        
# Check GPU availability
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

# Configure GPU memory growth
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = tf.config.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        print(e)
        
print("====================================================================")     
print("||                     DRIVERS AND GPU INFO                       ||")       
print("====================================================================")      
print("")

gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

# SYSTEM PARAMETERS

In [ ]:
import psutil

def get_ram_usage():
    return psutil.virtual_memory().used / (1024 ** 3) 

ram_usage_gb = get_ram_usage()
print(f"RAM Usage: {ram_usage_gb:.2f} GB")

# Get memory usage information
memory = psutil.virtual_memory()

# Total RAM in bytes
total_memory = memory.total

# RAM used in bytes
used_memory = memory.used

# RAM free in bytes
free_memory = memory.available

# RAM usage percentage
memory_percentage = memory.percent

print(f"Total Memory: {total_memory / (1024 ** 3)} GB")
print(f"Used Memory: {used_memory / (1024 ** 3)} GB")
print(f"Free Memory: {free_memory / (1024 ** 3)} GB")
print(f"Memory Usage: {memory_percentage}%")

print("")
print("====================================================================")
print("||                     SYSTEM INFO                                 ||")
print("====================================================================")
print("")

from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

# VERIFICATION

In [ ]:
import os
HOME = os.getcwd()
print("Working at: ", HOME)

In [ ]:
from IPython import display
display.clear_output()

import ultralytics
ultralytics.checks()

# 1. Creation of the first model

In [6]:
# Libraries for phase 1

from ultralytics import YOLO
from IPython.display import display, Image

# 1.1
import matplotlib.pyplot as plt
import numpy as np
import shutil

# 1.2
import os
from skimage import exposure
from PIL import Image

In [3]:
model = YOLO("yolov8m.pt")

data_yaml_path = 'data/yolo_train_dataset/data.yaml'

results = model.train(
    data=data_yaml_path,
    epochs=150,
    imgsz=960,
    batch=16,
    name='TrueDentYOLOv8m',
    lr0=0.01, lrf=0.01, warmup_epochs=3, patience=50,
    device=0,  # 0 for GPU, -1 for CPU
)

metrics = model.val()
print(metrics)

New https://pypi.org/project/ultralytics/8.3.144 available 😃 Update with 'pip install -U ultralytics'
Ultralytics 8.3.141 🚀 Python-3.11.2 torch-2.7.0+cu126 CUDA:0 (NVIDIA GeForce RTX 4060 Ti, 15956MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=data/yolo_train_dataset/data.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=150, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=960, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8m.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=TrueDentYOLOv8m4, nbs=64, nms=F

train: Scanning /home/mike/Desktop/codes/projects/AI_PRJ/TrueDent-AI/TrueDent/01_detection_model/model/data/yolo_train_dataset/train/labels.cache... 894 images, 0 backgrounds, 0 corrupt: 100%|██████████| 894/894 [00:00<?, ?it/s]

WARNING ⚠️ Box and segment counts should be equal, but got len(segments) = 384, len(boxes) = 23958. To resolve this only boxes will be used and all segments will be removed. To avoid this please supply either a detect or segment dataset, not a detect-segment mixed dataset.


val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 230.6±68.2 MB/s, size: 57.0 KB)


val: Scanning /home/mike/Desktop/codes/projects/AI_PRJ/TrueDent-AI/TrueDent/01_detection_model/model/data/yolo_train_dataset/valid/labels.cache... 64 images, 0 backgrounds, 0 corrupt: 100%|██████████| 64/64 [00:00<?, ?it/s]


...


107 epochs completed in 1.304 hours.
Optimizer stripped from /home/mike/Desktop/codes/projects/AI_PRJ/TrueDent-AI/runs/detect/TrueDentYOLOv8m4/weights/last.pt, 52.1MB
Optimizer stripped from /home/mike/Desktop/codes/projects/AI_PRJ/TrueDent-AI/runs/detect/TrueDentYOLOv8m4/weights/best.pt, 52.1MB

Validating /home/mike/Desktop/codes/projects/AI_PRJ/TrueDent-AI/runs/detect/TrueDentYOLOv8m4/weights/best.pt...
Ultralytics 8.3.141 🚀 Python-3.11.2 torch-2.7.0+cu126 CUDA:0 (NVIDIA GeForce RTX 4060 Ti, 15956MiB)
Model summary (fused): 92 layers, 25,858,288 parameters, 0 gradients, 78.8 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:01<00:00,  1.38it/s]


                   all         64       1825      0.822      0.859      0.881      0.672
                    11         58         58      0.871      0.897      0.862      0.611
                    12         60         60      0.848      0.839      0.826      0.553
                    13         58         58      0.782      0.828      0.828      0.624
                    14         57         57      0.815      0.773       0.86      0.607
                    15         53         53      0.801       0.83       0.87      0.605
                    16         54         54      0.872      0.881      0.887      0.726
                    17         62         62      0.893      0.811      0.904      0.683
                    18         50         50      0.793       0.84      0.869      0.669
                    21         61         61      0.854      0.852      0.853      0.604
                    22         59         59      0.824      0.847      0.825       0.61
                    2

val: Scanning /home/mike/Desktop/codes/projects/AI_PRJ/TrueDent-AI/TrueDent/01_detection_model/model/data/yolo_train_dataset/valid/labels.cache... 64 images, 0 backgrounds, 0 corrupt: 100%|██████████| 64/64 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 4/4 [00:02<00:00,  1.41it/s]


                   all         64       1825      0.822      0.858      0.881      0.673
                    11         58         58      0.872      0.897      0.863      0.612
                    12         60         60      0.848      0.838      0.826       0.56
                    13         58         58      0.784      0.828      0.829      0.625
                    14         57         57      0.818      0.772       0.86      0.613
                    15         53         53      0.807       0.83      0.871       0.61
                    16         54         54      0.872      0.881      0.887      0.728
                    17         62         62      0.893      0.811      0.905      0.683
                    18         50         50      0.798       0.84       0.87      0.668
                    21         61         61      0.858      0.852      0.852      0.608
                    22         59         59      0.824      0.847      0.827      0.617
                    2

In [9]:
pt_path = '/home/mike/Desktop/codes/projects/AI_PRJ/TrueDent-AI/runs/detect/TrueDentYOLOv8m4/weights/best.pt'

# Carpeta destino y nombre final del .onnx exportado
export_dir = 'final_models'
export_name = 'TrueDent_v1.onnx'

# Crear carpeta si no existe
os.makedirs(export_dir, exist_ok=True)

# Cargar el modelo
model = YOLO(pt_path)

# Exportar a ONNX (se guarda al lado del .pt por defecto)
exported_model_path = model.export(format='onnx')

# Mover el archivo exportado a la ruta deseada
onnx_generated = os.path.splitext(pt_path)[0] + '.onnx'
onnx_target_path = os.path.join(export_dir, export_name)

# Verificamos que se haya creado
if os.path.exists(onnx_generated):
    shutil.move(onnx_generated, onnx_target_path)
    print(f"✅ Modelo exportado a: {onnx_target_path}")
else:
    print("❌ No se encontró el archivo .onnx generado.")


Ultralytics 8.3.141 🚀 Python-3.11.2 torch-2.7.0+cu126 CPU (Intel Core(TM) i5-10400F 2.90GHz)
Model summary (fused): 92 layers, 25,858,288 parameters, 0 gradients, 78.8 GFLOPs

PyTorch: starting from '/home/mike/Desktop/codes/projects/AI_PRJ/TrueDent-AI/runs/detect/TrueDentYOLOv8m4/weights/best.pt' with input shape (1, 3, 960, 960) BCHW and output shape(s) (1, 36, 18900) (49.7 MB)

ONNX: starting export with onnx 1.17.0 opset 19...
ONNX: slimming with onnxslim 0.1.53...
ONNX: export success ✅ 2.0s, saved as '/home/mike/Desktop/codes/projects/AI_PRJ/TrueDent-AI/runs/detect/TrueDentYOLOv8m4/weights/best.onnx' (99.1 MB)

Export complete (3.5s)
Results saved to /home/mike/Desktop/codes/projects/AI_PRJ/TrueDent-AI/runs/detect/TrueDentYOLOv8m4/weights
Predict:         yolo predict task=detect model=/home/mike/Desktop/codes/projects/AI_PRJ/TrueDent-AI/runs/detect/TrueDentYOLOv8m4/weights/best.onnx imgsz=960  
Validate:        yolo val task=detect model=/home/mike/Desktop/codes/projects/AI_PRJ/

## 1.1 Data preparation

In [10]:
cls=[]
for result in results :
    boxes =result.boxes
    classes=boxes.cls
    cls.append(classes)

for i in range(len(cls)):
    cls[i]=cls[i].cpu().numpy()
  
box=[]

for result in results :
    boxes =result.boxes
    boxes=boxes.xywh
    box.append(boxes)

for i in range(len(box)):
    box[i]=box[i].cpu().numpy()

box[0]


TypeError: 'DetMetrics' object is not iterable

In [ ]:

# ==========================================================
#                 Convert to binary masks
# ==========================================================

bin_masks=[]
for j in range(len(box)):
   boxes1 = box[j]
   classes1=cls[j]
   boxes1_list=boxes1.tolist()
   classes1_list=classes1.tolist()
   binary_map = np.zeros((32,640,640), dtype=np.uint8)
   for box1, class1 in zip(boxes1_list,classes1_list):
       x,y,w,h=box1
       x1, y1 = round(x), round(y)
       x2, y2 = round(x+w), round(y+h)
       binary_map[int(class1), y1:y2, x1:x2] = 1

   bin_masks.append(binary_map)

del results, cls, box
bin_masks[0]

In [ ]:

# ==========================================================
#                 Convert to bounding boxes
# ==========================================================

def remove_nan(bb):
    bb=np.nan_to_num(bb, nan=0)
    return bb

bin_masks = list(map(remove_nan, bin_masks))

## 1.2 Pairing images with masks

In [ ]:
images_folder = 'data/yolo_train_dataset/train/images'
labels_folder = 'data/yolo_train_dataset/train/labels'

image_files = [f for f in os.listdir(images_folder) if f.endswith('.jpg')]
label_files = [f for f in os.listdir(labels_folder) if f.endswith('.txt')]

image_files.sort()
label_files.sort()

image_label_pairs = []

for image_file in image_files:
    image_path = os.path.join(images_folder, image_file)
    label_file = image_file.replace('.jpg', '.txt')
    label_path = os.path.join(labels_folder, label_file)

    if os.path.isfile(label_path):
        image_label_pairs.append((image_path, label_path))

for image_path, label_path in image_label_pairs:
    print("Image:", image_path)
    print("Label:", label_path)
    print()

del image_files, label_files

In [ ]:
def apply_clahe(image, clip_limit=0.02):
    image_float = image.astype(float) / 255.0
    image_clahe = exposure.equalize_adapthist(image_float, clip_limit=clip_limit)
    image_clahe = (image_clahe * 255).astype(image.dtype)

    return image_clahe

In [ ]:
image_data = []
mask_data = []  # bounding boxes (class_id, x_center, y_center, width, height)

for img_file, label_file in image_label_pairs:
    # Load image
    img = Image.open(img_file)
    img_data = np.array(img, dtype=np.float16)
    # Apply CLAHE if desired, otherwise comment out the following line
    img_data = apply_clahe(img_data)
    image_data.append(img_data)

    # Load bounding boxes from .txt
    with open(label_file, 'r') as f:
        bboxes = []
        for line in f:
            parts = line.strip().split()
            if len(parts) == 5:
                class_id, x_center, y_center, width, height = map(float, parts)
                bboxes.append((int(class_id), x_center, y_center, width, height))
        mask_data.append(bboxes)

In [ ]:
# ===========================================================
#                 check the paired data
# ===========================================================

dataset= list(zip(image_data,mask_data,bin_masks))

print("Images:", len(image_data))
print("Masks:", len(mask_data))
print("Bin Masks:", len(bin_masks))

# 2. Preparation of the dataset

In [ ]:
# Libraries for phase 2

import cv2
import numpy as np

# 2.1
import random

In [ ]:
def resize_img(input_image, input_mask, input_bb):
    input_image = cv2.resize(input_image, (512, 512), interpolation=cv2.INTER_NEAREST)

    # input_bb
    if input_bb.ndim == 3:
        input_bb = np.transpose(input_bb, axes=[1, 2, 0])
    input_bb = cv2.resize(input_bb, (512, 512), interpolation=cv2.INTER_NEAREST)

    # input_mask
    if input_mask.ndim == 3:
        input_mask = np.transpose(input_mask, axes=[1, 2, 0])
    input_mask = cv2.resize(input_mask, (512, 512), interpolation=cv2.INTER_NEAREST)

    return input_image, input_mask, input_bb

def augment(input_image,input_mask,input_bb):
    if np.random.uniform() > 0.5:
        input_image = np.fliplr(input_image)
        input_mask = np.fliplr(input_mask)
        input_bb = np.fliplr(input_bb)
        
    return input_image,input_mask,input_bb

def normalize(image_data):
    image_data = (image_data - np.min(image_data)) / (np.max(image_data) - np.min(image_data))
    return image_data

def load_image_train(datapoint):
    input_image = np.array(datapoint[0])
    input_mask = np.array(datapoint[1])
    input_bb = np.array(datapoint[2])
    input_image, input_mask, input_bb = resize_img(input_image, input_mask, input_bb)
    input_image = normalize(input_image)
    input_image, input_mask, input_bb = augment(input_image, input_mask, input_bb)


    return input_image, input_mask, input_bb

In [ ]:
new_dataset=list(map(load_image_train, dataset))

## 2.1 Divide by category (80% train, 20% test)

In [ ]:
# Define the dataset and categories
dataset = new_dataset
categories = [24, 72, 15, 32, 37, 30, 33, 140, 7, 35]

# Initialize the test and train datasets
test_dataset = []
train_dataset = []

# Set the random seed for reproducibility
random.seed(42)
start_idx = 0

print("Dataset length: ", len(dataset))

# Loop through each category
for i, category_count in enumerate(categories):
    end_idx = start_idx + category_count
    
    # Slice de la categoría
    category_data = dataset[start_idx:end_idx]
    
    # Shuffle dentro de esa categoría
    random.shuffle(category_data)
    
    # 20% para test
    num_test = int(0.2 * category_count)
    
    test_images = category_data[:num_test]
    train_images = category_data[num_test:]
    
    test_dataset.extend(test_images)
    train_dataset.extend(train_images)
    
    print(f'Category {i + 1}: {num_test} images in the test dataset')
    
    start_idx = end_idx

print(len(train_dataset))

# 3. Visualize masks

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from matplotlib.colors import ListedColormap

In [ ]:
obj=train_dataset[10]
image=obj[0]
mask=obj[1]
bb=obj[2]

pil_image = tf.keras.preprocessing.image.array_to_img(image)
plt.imshow(pil_image)
plt.axis('off')

In [ ]:
if mask.ndim == 2:
    mask = mask[..., np.newaxis]  # Convertir a (512, 512, 1)

colormap = ListedColormap(['#000000'] * 32)

combined_image = np.zeros((512,512, 3), dtype=np.uint16)

for i in range(mask.shape[2]):
    image = mask[:,:,i]

    if i == 0 or i == 1 or i == 8 or i == 9 or i == 16 or i == 17 or i == 24 or i == 25:
      colormap.colors[i] = '#2828a2'
    elif i == 2 or i == 10 or i == 18 or i == 26:
      colormap.colors[i] = '#09c5c5'
    elif i == 3 or i == 4 or i == 11 or i == 12 or i == 19 or i == 20 or i == 27 or i == 28:
      colormap.colors[i] = '#1caf1c'
    elif i == 5 or i == 6 or i == 7:
      colormap.colors[i] = '#eeee25'
    elif i == 13 or i == 14 or i == 15:
      colormap.colors[i] = '#eeee25'
    elif i == 21 or i == 22 or i == 23:
      colormap.colors[i] = '#eeee25'
    elif i == 29 or i == 30 or i == 31:
      colormap.colors[i] = '#eeee25'

    hex_color = colormap.colors[i]
    rgb_color = np.array([int(hex_color[i:i+2], 16) for i in (1, 3, 5)], dtype=np.uint16)

    mask_new = np.where(image > 0, 1, 0)

    rgb_color_tiled = np.tile(rgb_color, (512, 512, 1))

    colored_image = mask_new[:, :, np.newaxis] * rgb_color_tiled

    combined_image = combined_image + colored_image

    combined_image = combined_image.astype(np.uint8)

plt.imshow(combined_image)
plt.axis('off')
plt.show()

In [ ]:
colormap = ListedColormap(['#000000'] * 32)

combined_image = np.zeros((512,512, 3), dtype=np.uint16)

for i in range(bb.shape[2]):
    image = bb[:,:,i]

    if i == 0 or i == 1 or i == 8 or i == 9 or i == 16 or i == 17 or i == 24 or i == 25:
      colormap.colors[i] = '#2828a2'
    elif i == 2 or i == 10 or i == 18 or i == 26:
      colormap.colors[i] = '#09c5c5'
    elif i == 3 or i == 4 or i == 11 or i == 12 or i == 19 or i == 20 or i == 27 or i == 28:
      colormap.colors[i] = '#1caf1c'
    elif i == 5 or i == 6 or i == 7:
      colormap.colors[i] = '#eeee25'
    elif i == 13 or i == 14 or i == 15:
      colormap.colors[i] = '#eeee25'
    elif i == 21 or i == 22 or i == 23:
      colormap.colors[i] = '#eeee25'
    elif i == 29 or i == 30 or i == 31:
      colormap.colors[i] = '#eeee25'

    hex_color = colormap.colors[i]
    rgb_color = np.array([int(hex_color[i:i+2], 16) for i in (1, 3, 5)], dtype=np.uint16)

    mask_new = np.where(image > 0, 1, 0)

    rgb_color_tiled = np.tile(rgb_color, (512,512, 1))

    colored_image = mask_new[:, :, np.newaxis] * rgb_color_tiled

    combined_image = combined_image + colored_image

    combined_image = combined_image.astype(np.uint8)

plt.imshow(combined_image)
plt.axis('off')
plt.show()